In [ ]:
!pip install -q bitsandbytes accelerate peft datasets transformers trl


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.0/76.0 MB 8.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 491.2/491.2 kB 17.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 335.7/335.7 kB 14.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 7.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 183.9/183.9 kB 9.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 7.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 5.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 61.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 50.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 36.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 6.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [ ]:
from huggingface_hub import login
login()


Fine-Tuning LLAMA 2

In [ ]:


# STEP 3: Import Modules
import torch
from datasets import load_dataset
from transformers import (
    AutoTokenizer,
    AutoModelForCausalLM,
    BitsAndBytesConfig,
    TrainingArguments
)
from peft import LoraConfig, prepare_model_for_kbit_training
from trl import SFTTrainer

# STEP 4: Load Guanaco Dataset
dataset = load_dataset("mlabonne/guanaco-llama2", split="train[:100]")

# STEP 5: Model Setup with 8-bit Quantization + CPU Offloading
model_id = "NousResearch/Llama-2-7b-hf"

bnb_config = BitsAndBytesConfig(
    load_in_8bit=True,
    llm_int8_enable_fp32_cpu_offload=True
)

tokenizer = AutoTokenizer.from_pretrained(model_id, trust_remote_code=True)
tokenizer.pad_token = tokenizer.eos_token

model = AutoModelForCausalLM.from_pretrained(
    model_id,
    quantization_config=bnb_config,
    device_map="auto",
    trust_remote_code=True
)

# ✅ CRUCIAL FIX: Enable training support in 8-bit
model = prepare_model_for_kbit_training(model)

# ✅ Enable gradient checkpointing (memory saving)
model.gradient_checkpointing_enable()

# STEP 6: Tokenize Dataset with Smaller Max Length
def tokenize_function(example):
    return tokenizer(example["text"], truncation=True, padding="max_length", max_length=256)

tokenized_dataset = dataset.map(tokenize_function, batched=True)

# STEP 7: LoRA Configuration
lora_config = LoraConfig(
    r=8,
    lora_alpha=16,
    target_modules=["q_proj", "v_proj"],
    lora_dropout=0.05,
    bias="none",
    task_type="CAUSAL_LM"
)

# STEP 8: Training Arguments (optimized for T4 GPU)
training_args = TrainingArguments(
    output_dir="./llama2-guanaco-finetuned",
    per_device_train_batch_size=1,
    gradient_accumulation_steps=4,
    num_train_epochs=0.1,
    learning_rate=2e-4,
    fp16=True,
    logging_steps=10,
    save_strategy="epoch"
)

# STEP 9: SFTTrainer (LoRA + TRL)
trainer = SFTTrainer(
    model=model,
    train_dataset=tokenized_dataset,
    peft_config=lora_config,
    args=training_args
)

# STEP 10: Start Fine-Tuning
trainer.train()


Fetching 2 files:   0%|          | 0/2 [00:00<?, ?it/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/200 [00:00<?, ?B/s]

Map:   0%|          | 0/100 [00:00<?, ? examples/s]

Truncating train dataset:   0%|          | 0/100 [00:00<?, ? examples/s]

...

 ··········


wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: No netrc file found, creating one.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: sovanpal163 (sovanpal163-iit-patna) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


`use_cache=True` is incompatible with gradient checkpointing. Setting `use_cache=False`.
/usr/local/lib/python3.11/dist-packages/bitsandbytes/autograd/_functions.py:315: UserWarning: MatMul8bitLt: inputs will be cast from torch.float32 to float16 during quantization
  warnings.warn(f"MatMul8bitLt: inputs will be cast from {A.dtype} to float16 during quantization")


Step,Training Loss


TrainOutput(global_step=3, training_loss=2.7597996393839517, metrics={'train_runtime': 71.0138, 'train_samples_per_second': 0.141, 'train_steps_per_second': 0.042, 'total_flos': 121863866941440.0, 'train_loss': 2.7597996393839517})

In [ ]:
# Save & zip the fine-tuned model
model.save_pretrained("./llama2-guanaco-finetuned")
tokenizer.save_pretrained("./llama2-guanaco-finetuned")

!zip -r llama2-guanaco-finetuned.zip llama2-guanaco-finetuned


  adding: llama2-guanaco-finetuned/ (stored 0%)
  adding: llama2-guanaco-finetuned/special_tokens_map.json (deflated 72%)
  adding: llama2-guanaco-finetuned/tokenizer_config.json (deflated 68%)
  adding: llama2-guanaco-finetuned/checkpoint-13/ (stored 0%)
  adding: llama2-guanaco-finetuned/checkpoint-13/adapter_model.safetensors (deflated 8%)
  adding: llama2-guanaco-finetuned/checkpoint-13/scheduler.pt (deflated 56%)
  adding: llama2-guanaco-finetuned/checkpoint-13/special_tokens_map.json (deflated 78%)
  adding: llama2-guanaco-finetuned/checkpoint-13/tokenizer_config.json (deflated 68%)
  adding: llama2-guanaco-finetuned/checkpoint-13/scaler.pt (deflated 60%)
  adding: llama2-guanaco-finetuned/checkpoint-13/README.md (deflated 66%)
  adding: llama2-guanaco-finetuned/checkpoint-13/training_args.bin (deflated 52%)
  adding: llama2-guanaco-finetuned/checkpoint-13/tokenizer.model (deflated 55%)
  adding: llama2-guanaco-finetuned/checkpoint-13/tokenizer.json (deflated 85%)
  adding: llama